In [0]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('data', one_hot=True)

In [0]:
batch_size = 100
m_batch = mnist.train.num_examples//batch_size

def weight_variable(input_shape):
    initial = tf.truncated_normal(input_shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(input_shape):
    initial = tf.constant(0.1, shape=input_shape)
    return tf.Variable(initial)


def conv2d(input, weight):
    # input为4维数组，1表示batch_size，2表示输出的高度（in_height），
    # 3表示输出的宽度(in_weight)，最后1维表示输入图片的通道数(channel)
    # conv2d的第二个参数表示filter卷积核，filter[0]表示卷积核的高度, filter[1]表示
    # 卷积核的宽度，filter[2]表示输入的通道数(in_channel)，filter[3]表示输出的通道数
    # strides[0]=strides[3]=1，并且永远为1，strides[1]表示x方向的步长，strides[2]表示
    # y方向的步长
    return tf.nn.conv2d(
        input, filter=weight, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(input):
    return tf.nn.max_pool(input, ksize=[1, 2, 2, 1],
                strides=[1, 2, 2, 1], padding='SAME')


x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# 本身输入为28*28*1，卷积核为5*5,输出通道为32，padding，因此，输出为28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
# 池化为2*2, 步长为2，pandding，输出为14*14*32
h_pool1 = max_pool_2x2(h_conv1)

w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# 输入为14*14*32,卷积核为5*5,输出通道为64，panding,因此，输出为14*14*64
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
# 再进行步长为2的pool，输出结果为7*7*64
h_pool2 = max_pool_2x2(h_conv2)

w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

# 将4维输出扁平化
h_pool2_flatten = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flatten, w_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_dropout = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_dropout, w_fc2) + b_fc2)

corss_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
train = tf.train.AdamOptimizer(1e-4).minimize(corss_entropy)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(100):
        for batch in range(m_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs,y:batch_ys, keep_prob:0.5})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_prob:1.0})
        print('%d test accuracy %f' % (epoch, acc))